In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

ModuleNotFoundError: No module named 'azureml'

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


### YOUR CODE HERE ###

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive import normal, uniform, choice
import shutil, sys

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###

ps = RandomParameterSampling( {
        "--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
        "--max_iter": choice(50, 100, 150, 200, 250, 500)
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ### 
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1) # delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

if "outputs" not in os.listdir():
    os.makedirs('./outputs', exist_ok=True)

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
experiment_folder = "training"

shutil.copy('train.py', experiment_folder)


est = SKLearn(source_directory = experiment_folder,
                 entry_script = 'train.py',
                 compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy = policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)

In [ ]:

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print(best_run.get_file_names())
### YOUR CODE HERE ###
model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files(path=data_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x_df, y_df = clean_data(data) ### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= 'Accuracy',
    training_data= x_df,
    label_column_name= 'poutcome',
    n_cross_validations=5)

In [2]:
# Submit your automl run

run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.
best_model = run.get_output()
best_run, fitted_model = automl_run.get_output()
model = best_run.register_model(model_name='sklearn-automl', model_path='outputs/model.pkl')
### YOUR CODE HERE ###

In [ ]:
#  cpu_cluster.delete()